<a href="https://colab.research.google.com/github/victorviro/Big-Data/blob/main/Data_Partitioning_in_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✂️ Data partitioning in Apache Spark

**Distributed computing** does not only means the ⚙️ **processing of data in multiple nodes in a distributed manner**, but also the **data is stored distributedly in different nodes**. Each node is responsible for processing the data it stores. **Data ✂️ partitioning** plays a 🗝 critical role in the 🚀 performance of the processing of huge volumes of data in Spark.

To demonstrate Spark’s partitioning, we’ll walk through an **exercise** application of a **simple log dataset**. 

The table of contents of this notebook is as follow:

1. [ℹ️ Introduction](#1)
2. [🔀 Shuffle process](#2)
3. [📋 Exercise](#3)
    1. [📥 Running Pyspark in Colab](#3.1)
    2. [👀 Description of the data](#3.2)
    3. [🔢 Number of partitions](#3.3)
    4. [✂️ Repartition](#3.4)
        1. [Repartition by number of partitions](#3.4.1)
        2. [Coalesce](#3.4.2)
        3. [Repartition (by column)](#3.4.3)
            1. [`partitionBy`](#3.4.3.1)
4. [📄 Summary](#4)
5. [📕 References](#5)

# ℹ️ Introduction <a name="1"></a>




In Spark, as we explained in the notebook [Introduction to Spark](https://nbviewer.org/github/victorviro/Big-Data/blob/main/Introduction_to_Spark.ipynb), the data is distributed in the nodes or 👷‍♂️ workers. That is, **a dataset is distributed in several nodes through a technique called ✂️ "Partitioning"**.

- A **partition** of a dataset contains a **part of the dataset** and must be **stored in several nodes** (***replicas***) so that the availability of data is guaranteed in case of one node ❌ fails. 

- When a node 👎 fails or abandons the cluster, the partitions in that node are reassigned in a ⚖️ uniform manner across the remaining nodes. Similarly, when a node joins the cluster, the partitions must be rebalanced across the new machines.

<center><img src='https://i.ibb.co/2k21PKN/1-R81ph-LYl-QIAkx-Ia-LRsz3-EQ.png'></center>

In the image above, we have a dataset that is ✂️ divided into 8 partitions. For each partition, there are 3 replicas, and each replica is stored in 3 different nodes. If, for example, the *Node-4* ❌ fails, the data is available in other nodes.

- Each node must process the data that it stores and ➡️ deliver the result to another node. The node that receives this result must ⚙️ process this data and deliver the new result to another node. 

- A node may contain different partitions.

- When processing, Spark 🖇 assign a task to each partition, and each work subprocess only can process a task at a time.

A further visual explanation of partitioning is available in this [video](https://youtu.be/4Gfl0WuONMY?list=PLjH60bdMRScnn16R3cbb2CLxkbf0a9OO5) by Jesse Anderson.

There is a 👍👎 **tradeoff when partitioning datasets**. Having too ⬆ many partitions or too ⬇ few is not an ideal solution. The number of partitions in spark should be decided 🤔 thoughtfully based on the cluster 🔩 configuration and 📜 requirements of the application.

|                        | 👎 cons |   
|------------------------|---------|
| ⬇ **a few partitions** |It can cause the application does not use all available nodes in the cluster, causing work 🤯 overload in some nodes    |   
| ⬆ **a lot of partitions** | It increases ⬆ parallelism but it can provoke that each partition has ⬇ less data or no data at all causing Spark manages too many small tasks and hence increasing the processing ⏳ time due to the heavy 🔀 shuffle process | 

# 🔀 Shuffle process <a name="2"></a>

Each time a 🚌 [wide transformation](https://nbviewer.org/github/victorviro/Big-Data/blob/main/Introduction_to_Spark.ipynb#3.3) is executed in the processing, a node must collect the result of several nodes to continue the aggregation process. The 🔀 **shuffle process** occurs **between processing stages**. It tries to shuffle the results between nodes, but procuring that a single node has the necessary data to process ♻️ efficiently.

<center><img src='https://i.ibb.co/x22HWVY/1-EHYJ6-Ou5rv-QMz-Yl9-T0-JOv-A.png'></center>

In the example of the image below, we are 🔢 counting the number of records by each color. In the first stage (*Map*) the data of each node is processed. Each node sorts the records according to its color and it ➡️ delivers the result to the following node so that it continues the processing. Before delivering the results to the following node, the 🔀 *Shuffle* process happens. It makes sure a node has the records of a single color so that the node can count them, and in this way, the *Reduce* process delivers the final result.

- If data is partitioned in a lot of nodes, this implies a 💪 heavy Shuffle process. There would be a lot of data ➡️ sending across nodes, increasing the processing ⏳ time. This can be avoided by making a proper ✂️ repartition of the data. 

# 📋 Exercise <a name="3"></a> 

### 📥 Running Pyspark in Colab <a name="3.1"></a> 

To run spark in Colab, we need to first install all the dependencies, i.e. Apache Spark with Hadoop, Java 8, and Findspark to locate spark in the system.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xvf "spark-3.0.3-bin-hadoop3.2.tgz"
!pip install -q findspark

Now that we installed Spark and Java, we set the environment path which enables us to run Pyspark.

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

Finally, let's run a local spark session to test our installation:

In [6]:
!pip install pyspark==3.2.1

In [8]:
import findspark
findspark.init('/content/spark-3.0.3-bin-hadoop3.2')
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

print(spark)

In [ ]:
!sudo apt-get install tree

## 👀 Description of the data <a name="3.2"></a> 

The **dataset** we are going to use for the exercise contains **logs information** of an application. It's stored in a **single file** in parquet format. Let's 📥 download the file, load it in a PySpark dataframe, and display some records. 

In [10]:
data_path = "data"
!mkdir {data_path}
!wget -q https://github.com/victorviro/Big-Data/raw/main/files/logs2.snappy.parquet -P {data_path}/

In [11]:
logs = spark.read.parquet(data_path)
print(f'Number of records: {logs.count()}')
logs.show(10, False)

Number of records: 2000
+----------+------------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|date      |time        |severity|message                                                                                                                                                                |
+----------+------------+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2019-01-25|18:01:47,978|INFO    |Created MRAppMaster for application appattempt_1445144423722_0020_000001                                                                                               |
|2016-10-07|18:01:48,963|INFO    |Executing with tokens:                                                                                                            

In [12]:
logs.printSchema()

root
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- severity: string (nullable = true)
 |-- message: string (nullable = true)



We have a dataframe with 2000 records and 4 columns: *date*, *time*, *severity*, and *message*. 

| Column name | Description                      |   
|-------------|----------------------------------|
| date        | The 📅 date of the log             |            
| time        | The ⌚️ time of the log            |    
| severity    | The 🎚 level of severity of the log |   
| message     | The 💬 text content of the log     |           

## 🔢 Number of partitions <a name="3.3"></a> 

Let's see the number of partitions of the dataframe with the `getNumPartitions` method.

In [13]:
print(logs.rdd.getNumPartitions())

1


In our case, we are reading a single small file from our local disk so the 🔢 number of partitions is 1. However, if we read huger datasets, the number of partitions will ⬆ increase. For example, 
- If we read a huge dataset from HDFS,  Spark, by default, creates one partition for each block of the file.
- If we create a dataframe instead of reading it, the number of partitions of that dataset is defined by the `sc.defaultParallelism` 🔩 configuration value. The default value for this configuration is set to the number of all cores on all nodes in the cluster. On local, it is set to the number of cores on our system.

We can store this dataframe in HDFS, S3, local disk... Let's store the dataframe on local disk with the following command:

In [14]:
output_path = "output_data/"
logs.write.mode("overwrite").parquet(output_path)

In this way the dataframe will be stored in local disk. Let's see the path where the dataframe was stored:

In [15]:
!ls {output_path}

part-00000-48587dcb-b07e-4b5e-95f3-91663f9f3b8f-c000.snappy.parquet  _SUCCESS


The dataframe is stored in 1 partition. If we load that single partition, it will contain all records of the dataset:

In [16]:
dfpart00000 = spark.read.parquet(output_path +"/*part-00000*")
dfpart00000.count()

2000

## ✂️ Repartition <a name="3.4"></a> 

### Repartition by number of partitions <a name="3.4.1"></a> 

If we want to ⬆ **increase the number of partitions** so that more nodes can ⚙ process these partitions, we can use the function [**`repartition`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.repartition.html). It **performs a 🔀 shuffle operation** to reassign the data to new partitions. Let's redistribute the dataframe into 10 partitions and store it. 

In [17]:
logs = logs.repartition(10)
logs.write.mode("overwrite").parquet(output_path)
print(logs.rdd.getNumPartitions())

10


Let's see the path where the dataframe was stored:

In [18]:
!ls {output_path}

part-00000-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00001-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00002-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00003-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00004-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00005-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00006-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00007-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00008-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
part-00009-c40203b0-22a2-4c7c-a2e1-26dc4191a567-c000.snappy.parquet
_SUCCESS


A new hash code was created for the dataframe. Note the **hash** in the name of the partitions **is the same**, meaning they are **partitions of the same dataframe**. Now each partition contains $\frac{1}{10}$ part of the data (200 records).

In [19]:
dfpart00001 = spark.read.parquet(f'{output_path}/*part-00001*')
dfpart00001.count()

200

This will allow a single node to process less data, but it may ⬆ increase the processing ⏳ time of the 🔀 shuffle operation.

### Coalesce <a name="3.4.2"></a> 

The method [**`coalesce`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.coalesce.html) is used to ⬇ **decrease the number of partitions**. It does **not** perform the **🔀 shuffle operation**, so that it can't shuffle the data to new nodes. It can send data to nodes that already have part of the dataframe stored.

If we try to repartition to ⬆ more nodes with this function, the current number of partitions will be kept:

In [20]:
logs = logs.coalesce(20)
print(logs.rdd.getNumPartitions())

10


Let's see what happens if we try to ⬇ decrease the number of partitions with the `coalesce` method.

In [21]:
logs = logs.coalesce(5)
logs.write.mode("overwrite").parquet(output_path)
print(logs.rdd.getNumPartitions())

5


In [22]:
!ls {output_path}

part-00000-551196b8-c094-4d29-8b05-5b2613bd0957-c000.snappy.parquet
part-00001-551196b8-c094-4d29-8b05-5b2613bd0957-c000.snappy.parquet
part-00002-551196b8-c094-4d29-8b05-5b2613bd0957-c000.snappy.parquet
part-00003-551196b8-c094-4d29-8b05-5b2613bd0957-c000.snappy.parquet
part-00004-551196b8-c094-4d29-8b05-5b2613bd0957-c000.snappy.parquet
_SUCCESS


Each partition will contain the $\frac{1}{5}$ part of the records (400).

In [23]:
dfpart00001 = spark.read.parquet(f'{output_path}/*part-00001*')
dfpart00001.count()

400

⬇ Decreasing the number of partitions, a node must process more data, but the 🔀 shuffle process will take less time.

### Repartition (by column) <a name="3.4.3"></a> 

It's possible to use the function [**`repartition`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.repartition.html) to sort the table. It's necessary to 👉 identify a column for which group the data in a way so that a node has the data necessary for processing and the 🔀 shuffle operation is minimum.

Suppose we want to group by "severity", it will be convenient to partition the data for each level of severity and thus ♻️ optimize the ⚙ processing more than having the data partitioned by simply a number of partitions defined.

This also allows for making **queries more efficiently**. If we want to filter the data for a value of the field "severity" we would **not have to read the whole dataframe**, only a partition that contains the data for that 🎚 level of severity. This function uses the 🔀 **shuffle operation** to distribute the data.

In [24]:
logs = logs.repartition("severity")
logs.write.mode("overwrite").parquet(output_path)
print(logs.rdd.getNumPartitions())

200


By default, Spark creates a minimum of 200 partitions, but in our case, only 5 files contain the information for each 🎚 level of severity. If it would be 10 levels of severity, it whould be 10 files with information.

In [25]:
!ls -l --block-size=M {output_path}

total 1M
-rw-r--r-- 1 root root 1M May  2 10:48 part-00000-10c2114f-1909-41d4-b979-79b1c2684aa4-c000.snappy.parquet
-rw-r--r-- 1 root root 1M May  2 10:48 part-00004-10c2114f-1909-41d4-b979-79b1c2684aa4-c000.snappy.parquet
-rw-r--r-- 1 root root 1M May  2 10:48 part-00015-10c2114f-1909-41d4-b979-79b1c2684aa4-c000.snappy.parquet
-rw-r--r-- 1 root root 1M May  2 10:48 part-00037-10c2114f-1909-41d4-b979-79b1c2684aa4-c000.snappy.parquet
-rw-r--r-- 1 root root 1M May  2 10:48 part-00095-10c2114f-1909-41d4-b979-79b1c2684aa4-c000.snappy.parquet
-rw-r--r-- 1 root root 0M May  2 10:48 _SUCCESS


In [28]:
dfpart00004 = spark.read.parquet(f'{output_path}/*part-00004*')
dfpart00004.show(5)

+----------+------------+--------+--------------------+
|      date|        time|severity|             message|
+----------+------------+--------+--------------------+
|2019-01-25|18:01:47,978|    INFO|Created MRAppMast...|
|2016-10-07|18:01:48,963|    INFO|Executing with to...|
|2017-10-13|18:01:48,963|    INFO|                Kind|
|2016-05-10|18:01:49,228|    INFO|Using mapred newA...|
|2017-06-18|18:01:50,353|    INFO|OutputCommitter s...|
+----------+------------+--------+--------------------+
only showing top 5 rows



In [29]:
dfpart00037 = spark.read.parquet(f'{output_path}/*part-00037*')
dfpart00037.show(5)

+----------+------------+--------+--------------------+
|      date|        time|severity|             message|
+----------+------------+--------+--------------------+
|2017-02-14|18:05:27,570|    WARN|Address change de...|
|2016-12-19|18:05:27,570|    WARN|Failed to renew l...|
|2016-04-13|18:05:28,570|    WARN|Address change de...|
|2016-05-12|18:05:28,570|    WARN|Failed to renew l...|
|2017-02-06|18:05:29,570|    WARN|Address change de...|
+----------+------------+--------+--------------------+
only showing top 5 rows



We must **use this function carefully** as a lot of unnecessary partitions will be created.

In general, the partition by column is used to partition by multiple columns. Let's create 3 new columns: *Year, Month y Day*.

In [30]:
import pyspark.sql.functions as F
logs = logs.withColumn(
    "Year", F.year("date")).withColumn(
    "Month", F.month("date")).withColumn(
    "Day", F.dayofmonth("date")
)
logs = logs.repartition("Year", "Month", "Day", "severity")
logs.write.mode("overwrite").parquet(output_path)
print(logs.rdd.getNumPartitions())

200


In this way, 200 partitions are created anyway, but the data is distributed across all partitions. This can imply 🚀 performance 🐛 issues cause we will have **a lot of partitions with little data**.

In [ ]:
!ls -l --block-size=K {output_path}

To improve this, the method [**`partitionBy`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.partitionBy.html) is used.

#### [**`partitionBy`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.partitionBy.html) <a name="3.4.3.1"></a> 

The method [**`partitionBy`**](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrameWriter.partitionBy.html) ✂️ partitions the data in 📁 **folders according** to the ↕️ hierarchy of the **columns**, and thus **optimizing the query processes**.

In [31]:
logs.coalesce(1).write.partitionBy("severity").mode("overwrite").parquet(output_path)
logs.show(5, False)

+----------+------------+--------+------------------------------------------------------------------------+----+-----+---+
|date      |time        |severity|message                                                                 |Year|Month|Day|
+----------+------------+--------+------------------------------------------------------------------------+----+-----+---+
|2016-08-26|18:01:50,666|INFO    |Default file system [hdfs://msra-sa-41:9000]                            |2016|8    |26 |
|2016-08-26|18:01:50,728|INFO    |Emitting job history data to the timeline server is not enabled         |2016|8    |26 |
|2016-08-26|18:01:53,885|INFO    |task_1445144423722_0020_m_000009 Task Transitioned from NEW to SCHEDULED|2016|8    |26 |
|2015-12-11|18:01:57,291|INFO    |Opening proxy                                                           |2015|12   |11 |
|2016-02-09|18:03:03,795|INFO    |Progress of TaskAttempt attempt_1445144423722_0020_m_000002_0 is        |2016|2    |9  |
+----------+----

The dataframe will keep its structure, but if we see the file system: **a 📁 subdirectory is aggregated for each level of** severity, **the field that was used when partitioning**.

In [32]:
!tree {output_path}

output_data/
├── severity=ERROR
│   └── part-00000-f6bb18f9-aaeb-468b-9e29-5a29dc4f388d.c000.snappy.parquet
├── severity=FATAL
│   └── part-00000-f6bb18f9-aaeb-468b-9e29-5a29dc4f388d.c000.snappy.parquet
├── severity=INFO
│   └── part-00000-f6bb18f9-aaeb-468b-9e29-5a29dc4f388d.c000.snappy.parquet
├── severity=WARN
│   └── part-00000-f6bb18f9-aaeb-468b-9e29-5a29dc4f388d.c000.snappy.parquet
└── _SUCCESS

4 directories, 5 files


To 🚫 avoid a ⬆ huge number of partitions, we first have re-partitioned to a smaller number of partitions (1) and then partition by column again. So each 📁 subdirectory has 1 partition.

In practice, the most **common way to partition data by columns** is partitioning by 📅 **dates**.

In [33]:
logs.coalesce(1).write.partitionBy(
    "Year","Month","severity"
).mode("overwrite").parquet(output_path)
logs.show(5, False)

+----------+------------+--------+------------------------------------------------------------------------+----+-----+---+
|date      |time        |severity|message                                                                 |Year|Month|Day|
+----------+------------+--------+------------------------------------------------------------------------+----+-----+---+
|2016-08-26|18:01:50,666|INFO    |Default file system [hdfs://msra-sa-41:9000]                            |2016|8    |26 |
|2016-08-26|18:01:50,728|INFO    |Emitting job history data to the timeline server is not enabled         |2016|8    |26 |
|2016-08-26|18:01:53,885|INFO    |task_1445144423722_0020_m_000009 Task Transitioned from NEW to SCHEDULED|2016|8    |26 |
|2015-12-11|18:01:57,291|INFO    |Opening proxy                                                           |2015|12   |11 |
|2016-02-09|18:03:03,795|INFO    |Progress of TaskAttempt attempt_1445144423722_0020_m_000002_0 is        |2016|2    |9  |
+----------+----

In [34]:
!tree {output_path}

output_data/
├── _SUCCESS
├── Year=2015
│   ├── Month=10
│   │   ├── severity=ERROR
│   │   │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   │   ├── severity=INFO
│   │   │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   │   └── severity=WARN
│   │       └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   ├── Month=11
│   │   ├── severity=ERROR
│   │   │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   │   ├── severity=INFO
│   │   │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   │   └── severity=WARN
│   │       └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│   └── Month=12
│       ├── severity=ERROR
│       │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│       ├── severity=INFO
│       │   └── part-00000-7259f3e4-a1e5-4b3b-bc0b-745ccff44818.c000.snappy.parquet
│       └── severity=WARN


This distributes the data ♻️ **efficiently when we query filtering by these fields**. It will **not be necessary to read the whole dataset ** to process it, it will be enough to filter the fields that are required, and this optimizes the queries and the ⌛ time of the 🔀 shuffle operation.

If, for example, we want only the data for the year 2019, of month 2 and severity WARN, we will write the query in the following manner:

In [35]:
df2 = spark.read.parquet(output_path + "/Year=2019/Month=2/severity=WARN/")
df2.show(5, False)

+----------+------------+-------------------------------------------------------------------------------------------------------+---+
|date      |time        |message                                                                                                |Day|
+----------+------------+-------------------------------------------------------------------------------------------------------+---+
|2019-02-28|18:05:50,680|Failed to renew lease for [DFSClient_NONMAPREDUCE_1537864556_1] for 53 seconds.  Will retry shortly ...|28 |
+----------+------------+-------------------------------------------------------------------------------------------------------+---+



Even we can use the trick `*` to amplify the range of the query. Let's query the information for all year 2019 for the severity WARN:

In [36]:
df2 = spark.read.parquet(output_path + "/Year=2019/Month=*/severity=WARN/")
df2.show(5, False)

+----------+------------+--------------------------------------------------------------------------------------------------------+---+
|date      |time        |message                                                                                                 |Day|
+----------+------------+--------------------------------------------------------------------------------------------------------+---+
|2019-03-12|18:07:11,267|Failed to renew lease for [DFSClient_NONMAPREDUCE_1537864556_1] for 133 seconds.  Will retry shortly ...|12 |
|2019-05-07|18:10:22,013|Failed to renew lease for [DFSClient_NONMAPREDUCE_1537864556_1] for 324 seconds.  Will retry shortly ...|7  |
|2019-06-20|18:07:40,425|Failed to renew lease for [DFSClient_NONMAPREDUCE_1537864556_1] for 162 seconds.  Will retry shortly ...|20 |
|2019-02-28|18:05:50,680|Failed to renew lease for [DFSClient_NONMAPREDUCE_1537864556_1] for 53 seconds.  Will retry shortly ... |28 |
|2019-04-12|18:07:45,425|Address change detected. Old  

# 📄 Summary <a name="4"></a>

By using ✂️ partitions, the parellized use of the Spark cluster is ⬆ maximized, and the storing space is ⬇ reduced to achieve better 🚀 performance.

When 🤔 designing a **strategy** of partitioning (write partitions to a file system), we must take care of the access routes. For example, **is it common the usage of our partition keys (fields) in the filters?** Answering these ❓ questions helps to determine by which column we must partition.

However, partition does not mean, the more the best. **Spark recommends 2-3 tasks per each kernell of CPU** in the cluster. For instance, if we have 1000 kernells of CPU in our cluster, the recommended number of partitions is 2000-3000. Sometimes it depends on the distribution and asymmetry of the raw data. It must be adjusted to find the proper partitioning strategy. A proper partitioning strategy optimizes the query processes, grouping, and joining of datasets.

# 📕 References <a name="5"></a>

- [Spark SQL Documentation](https://spark.apache.org/docs/latest/sql-programming-guide.html)

- [Spark partitioning Understanding, sparkbyexamples](https://sparkbyexamples.com/spark/spark-partitioning-understanding/)

- [Understanding HDFS using Legos](https://youtu.be/4Gfl0WuONMY?list=PLjH60bdMRScnn16R3cbb2CLxkbf0a9OO5)